<span style="display:block;text-align:center;margin-right:105px"><img src="../media/logos/hacks-logo.png" width="200"/></span>

In [1]:
#!pip install cadCAD==0.4.23 &> /dev/null
#!pip uninstall numpy
#!pip uninstall pandas

#!pip install --user numpy
#!pip install --user  pandas

---

# Dependencies

In [5]:
# cadCAD standard dependencies

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD global simulation configuration list
from cadCAD import configs

# Included with cadCAD
import pandas as pd

In [6]:
# Additional dependencies

# For parsing the data from the API
import json
# For downloading data from API
import requests as req
# For generating random numbers
import math
# For visualization
import plotly.express as px

# For probability distributions
import scipy.stats as st

# Setup / Preparatory Steps

## Download historical prices from Coingecko API

In [7]:
def get_eth_btc_exchange(start_date: str,
                         end_date: str) -> pd.Series:
    """
    Get a Pandas DataFrame with 'timestamp' and 'eth_per_btc' columns, filtered
    between the `start_date` and `end_date` range.
    """
    # Link for the CoinGecko API
    # Documentation: https://www.coingecko.com/api/documentations/v3#/coins/get_coins__id__market_chart
    API_URI = 'https://api.coingecko.com/api/v3/coins/ethereum/market_chart?vs_currency=btc&days=max'
    
    # Download data from the API using the HTTP GET method
    api_response = req.get(API_URI)
    
    # Get the raw JSON string from the API response
    raw_json: str = api_response.content
        
    # Parse it into a Python dictionary 
    raw_price_data: dict = json.loads(raw_json)
        
    # Get the 'prices' content
    price_dict: dict = raw_price_data['prices']
    
    # Function for converting UNIX epochs into Python datetime
    convert_epochs_to_timestamp = lambda df: pd.to_datetime(df.timestamp, unit='ms')
    
    price_df = (pd.DataFrame(price_dict) # Generate a DataFrame from the price dictionary
                  .rename(columns={0: 'timestamp', 1: 'eth_per_btc'}) # Rename columns
                  .assign(timestamp=convert_epochs_to_timestamp) # Parse the provided epochs using lambda function
                  .query(f'timestamp >= "{start_date}"') # Filter so that we have only data after `start_date`
                  .query(f'timestamp <= "{end_date}"') # Filter so that we have only data before `end_date`
                  .reset_index(drop=True) # Reset the DataFrame indices
               )
    
    # Return the DataFrame to the user
    return price_df

# Store the ETH/BTC exchange prices for 2020
price_df = get_eth_btc_exchange('2020-01-01', '2021-01-01')
print(price_df)

     timestamp  eth_per_btc
0   2020-01-01     0.017954
1   2020-01-02     0.018138
2   2020-01-03     0.018247
3   2020-01-04     0.018285
4   2020-01-05     0.018242
..         ...          ...
362 2020-12-28     0.026061
363 2020-12-29     0.026997
364 2020-12-30     0.026833
365 2020-12-31     0.026110
366 2021-01-01     0.025458

[367 rows x 2 columns]


# Modelling

## 1. State Variables

In [8]:
initial_state = {
    'timestamp': None, # Date for each timestep
    'daily_price': 0.0, # Current price for ETH / BTC
    'eth_per_btc_change': None, # How much the ETH / BTC price has changed since the last day
}
initial_state

{'daily_price': 0.0, 'eth_per_btc_change': None, 'timestamp': None}

## 2. System Parameters

In [9]:
# Transform the price dataframe into a 
# {timestep: value} dictionary
exchange_per_timestep = price_df.to_dict(orient='index')

system_params = {
    'daily_eth_per_btc': [exchange_per_timestep],
}
exchange_per_timestep[5]

{'eth_per_btc': 0.018385021180886967,
 'timestamp': Timestamp('2020-01-06 00:00:00')}

## 3. Policy Functions

In [10]:
def p_exchange(params, substep, state_history, previous_state):
    """
    Get the daily price associated with a timestep
    """
    # Get the current timestep
    timestep = previous_state['timestep']
    
    # Get the values asssociated with the timestep key
    current_exchange = params['daily_eth_per_btc'][timestep]
    
    # Retrieve timestamp and daily price
    timestamp = current_exchange['timestamp']
    daily_price = current_exchange['eth_per_btc']
    
    return {'timestamp': timestamp,
            'daily_price': daily_price}

## 4. State Update Functions

In [12]:
def s_timestamp(params, substep, state_history, previous_state, policy_input):
    """
    Set the timestamp variable
    """
    value = policy_input['timestamp']
    
    return ('timestamp', value)


def s_daily_price(params, substep, state_history, previous_state, policy_input):
    """
    Set the daily price variable
    """
    value = policy_input['daily_price']
    
    return ('daily_price', value)


def s_eth_per_btc_change(params, substep, state_history, previous_state, policy_input):
    """
    Set the eth_per_btc_change variable as the difference
    between the current and previous daily prices
    """
    past_value = previous_state['daily_price']
    today_value = policy_input['daily_price']
    
    change = today_value - past_value
    
    return ('eth_per_btc_change', change)

## 5. Partial State Update Blocks

In [13]:
partial_state_update_blocks = [
    {
        'policies': {
            'p_exchange': p_exchange
        },
        'variables': {
            'timestamp': s_timestamp,
            'daily_price': s_daily_price,
            'eth_per_btc_change': s_eth_per_btc_change
        }
    }
]

# Simulation

## 6. Configuration

In [38]:
sim_config = config_sim({
    "N": 1, # the number of times we'll run the simulation ("Monte Carlo runs")
    "T": range(len(price_df)), # the number of timesteps the simulation will run for
    "M": system_params # the parameters of the system
})

In [39]:
del configs[:] # Clear any prior configs

In [40]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

## 7. Execution

In [41]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (367, 1, 1, 3)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.03s


## 8. Output Preparation

In [42]:
simulation_result = pd.DataFrame(raw_result)
simulation_result.head()

,timestamp,daily_price,eth_per_btc_change,simulation,subset,run,substep,timestep
0,NaT,0.000000,NaN,0,0,1,0,0
1,2020-01-01,0.017954,0.017954,0,0,1,1,1
2,2020-01-02,0.018138,0.000185,0,0,1,1,2
3,2020-01-03,0.018247,0.000109,0,0,1,1,3
4,2020-01-04,0.018285,0.000039,0,0,1,1,4


## 9. Analysis

In [43]:
pd.options.plotting.backend = "plotly"

In [44]:
# Plot the `daily_price` for each `timestamp` as a line plot
simulation_result.plot(
    kind='line',
    x='timestamp',
    y=['daily_price'],
    title="Daily ETH per BTC Price",
    labels={
        "value": "ETH per BTC",
    },
)

In [45]:
# Plot the `eth_per_btc_change` for each `timestamp` as a line plot
simulation_result.query('timestep > 1').plot(
    kind='line',
    x='timestamp',
    y=['eth_per_btc_change'],
    title="Daily ETH per BTC Price Change",
    labels={
        "value": "ETH per BTC Price Change",
    },
)

In [46]:
# Gamma distribution generator
def gamma_proc(size):
    return st.gamma.rvs(10, loc=0.012, scale=0.0015, size=size)

# Append gamma random points to the dataframe
def generate_gamma(df):
    new_df = (df.assign(origin='generated')
                .assign(daily_price=lambda df: gamma_proc(len(df)))
             )
    
    return pd.concat([df, new_df])

# Data pipeline
fig_df = (simulation_result.query("timestep > 1")
                           .assign(origin='data')
                           .pipe(generate_gamma)
         )

# Visualize results
px.histogram(fig_df,
             x='daily_price',
             marginal='violin',
             color='origin')

In [47]:
fig_df.plot(
    kind='line',
    x='timestamp',
    y=['daily_price'],
    title="Daily ETH per BTC Price",
    labels={
        "value": "ETH per BTC Price",
    },
    color='origin',
)

In [48]:
# Normal distribution generator
def normal_proc(size):
    return st.norm.rvs(loc=0.0, scale=0.02, size=size)

# Append normal random points to the dataframe
def generate_normal(df):
    new_df = (df.assign(origin='generated')
                .assign(normed_daily_price_change=lambda df: normal_proc(len(df)))
             )
    
    return pd.concat([df, new_df])

# Data pipeline
fig_df = (simulation_result.query("timestep > 1")
                           .assign(origin='data')
                           .assign(normed_daily_price_change=lambda df: df.daily_price.diff() / df.daily_price)
                           .pipe(generate_normal)
         )

# Visualize results
px.histogram(fig_df,
             x='normed_daily_price_change',
             marginal='violin',
             color='origin')

In [49]:
fig_df.plot(
    kind='line',
    x='timestamp',
    y=['normed_daily_price_change'],
    title="Daily ETH per BTC Price Change",
    labels={
        "value": "ETH per BTC Price Change",
    },
    color='origin',
)